In [1]:
import mwu_measures
import pandas as pd
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher
mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
this_corpus = mwu_measures.processing_corpus.get_processed_corpus('bnc', 'small_corpus.txt', chunk_size=10000000, verbose=False)

# this_corpus = mwu_measures.processing_corpus.process_corpus(test_corpus=True)

# ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)


# bigrams = [['b', 'd'], ['c', 'b'], ['a', 'c']]

helper = Fetcher(this_corpus)

In [2]:
ngrams = mwu_examples['Expression'].tolist()
helper.create_scores(ngrams)

In [3]:
test_weights = {'token_freq': 1/8, 'dispersion': 1/4, 'type_1': 1/16, 'type_2': 1/16, 'entropy_1': 1/16, 'entropy_2': 1/16, 'fw_assoc': 1/8, 'bw_assoc': 1/4}

In [4]:
ngrams = ['come in handy', 'family friend', 'database management system', 'one hundred percent', 'line of control', 'like a', 'boiled potatoes']

In [5]:
self = helper
normalized=True
self.get_score_batch(ngrams)

,ngram,comp_1,comp_2,mwu_score,ngram_length
0,boiled potatoes,boiled,potatoes,0.262146,2
1,one hundred,one,hundred,0.477816,2
2,family friend,family,friend,0.321364,2
3,database management,database,management,0.313036,2
4,like a,like,a,0.553499,2
5,come in,come,in,0.493726,2
6,line of,line,of,0.506568,2
7,line of control,line of,control,0.290103,3
8,come in handy,come in,handy,0.328495,3
9,database management system,database management,system,0.317768,3


In [7]:

sentence = 'Hello! Come in and eat boiled potatoes they are one hundred percent good.\nI remember when we met in the year 2000'
sentence = mwu_measures.process_text(sentence)

In [10]:
self.create_scores(sentence)

In [11]:
self.get_score_batch(sentence)

,ngram,comp_1,comp_2,mwu_score,ngram_length
0,boiled potatoes,boiled,potatoes,0.253083,2
1,boiled potatoes,boiled,potatoes,0.253083,2
2,hundred percent,hundred,percent,0.367901,2
3,hundred percent,hundred,percent,0.367901,2
4,in and,in,and,0.577979,2
5,in and,in,and,0.577979,2
6,we met,we,met,0.410347,2
7,we met,we,met,0.410347,2
8,remember when,remember,when,0.384407,2
9,remember when,remember,when,0.384407,2


In [ ]:
# TODO: Rewrite documentation and merge onto main. This is basically ready.
# TODO: Brown as default corpus.
# TODO: 4-grams. Should need minimal modification. Challenge might be RAM. Consider implementing an option to work from disk with duckdb?
# TODO: batches to not crash memory when using complete corpus. Worth it? faster because of the simpler joins? worth it to hash? Maybe a way to add rows to a table? Test whether it crashes with a long list of ngrams or with CoCA in the future.